In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import notebook
%matplotlib inline
matplotlib.style.use('ggplot')

In [2]:
def makemonth(data):
        
    month = []
    for i in notebook.tqdm(range(len(data.index))):
        for j in range(len(temp_rebalancing)):
            if data.index[i] == temp_rebalancing[j]:
                month.append(data.iloc[i])
                
    month_data = pd.DataFrame(month)
    
    return month_data

In [3]:
#데이터 불러오기

industry = pd.read_csv("fnguide_industry_group.csv",parse_dates = ['date'], low_memory = False)
industry.set_index('date', inplace = True)

mkt_cap = pd.read_csv("market_cap.csv",parse_dates = ['date'], low_memory = False)
mkt_cap.set_index('date', inplace = True)
mkt_cap = mkt_cap.shift(1) #mkt_cap 1기 전이 기준

rtn = pd.read_csv("rtn.csv",parse_dates = ['date'], low_memory = False)
rtn.set_index('date', inplace = True)
#rtn = pd.read_csv("return.csv",parse_dates = ['date'], low_memory = False)
#rtn.set_index('date', inplace = True)
#rtn = rtn.applymap(lambda x:pd.to_numeric(str(x).replace(",","")) if not pd.isna(x) else np.nan)
#rtn = rtn / 100

In [4]:
temp_rebalancing = []
for j in range(len(mkt_cap.index)-1):
    if mkt_cap.index.month[j] != mkt_cap.index.month[j+1]:
        temp_rebalancing.append(mkt_cap.index[j+1])

In [5]:
#공통인자만 추출
tic = list(set.intersection(set(industry.columns), set(mkt_cap.columns)))
tic = list(set.intersection(set(tic), set(rtn.columns)))
tic.sort()

In [6]:
#2000년대 이후만 남기기
industry = industry[tic].loc['2000-01-01':'2020-07-27'].iloc[1:]
mkt_cap = mkt_cap[tic].loc['2000-01-01':'2020-07-27'].iloc[1:]
rtn = rtn[tic].loc['2000-01-01':'2020-07-27'].iloc[1:]

In [7]:
month_mkt_cap = makemonth(mkt_cap)

In [8]:
#산업 리스트
inds = ["Hardware", "Capital Goods", "Materials", "Software", "Consumer Durables & Apparel", "Automobiles & Components", "Pharmaceuticals & Biotechnology", "Diversified Financial Services", "Semiconductors", "Food, Beverage & Tobacco", "Display", "Media", "Health Care Equipment Services", "Energy", "Household & Personal Products", "Consumer Services", "Retailing", "Transportation", "Commercial Services & Supplies", "Capital Markets", "Insurance", "Utility", "Telecommunication Services", "Real Estate"]

In [9]:
#한 기 전 mkt_cap 사용
month_mkt_cap = month_mkt_cap.shift(1).iloc[1:]

In [10]:
#월초값으로 ffill한 mkt_cap
month_mkt_cap = month_mkt_cap.fillna(10000)
temp = pd.DataFrame(index=mkt_cap.index, columns=mkt_cap.columns)
temp.loc[month_mkt_cap.index, :] = month_mkt_cap.values
temp = temp.fillna(method='ffill')
re_mkt_cap = temp.replace(10000, np.nan)
re_mkt_cap = re_mkt_cap.dropna(how = "all")

In [11]:
industry = industry.loc['2000-03-01':'2020-07-27']
rtn = rtn.loc['2000-03-01':'2020-07-27']

In [13]:
#weight 추출

#변수 설정
tmp_inds = [[] for _ in range(24)]
tmp_rtn = [[] for _ in range(24)]

#함수 설정
div = lambda x : x / x.sum()

#weight 추출
for i in notebook.tqdm(range(24)):
    tmp_inds[i] = re_mkt_cap[industry==inds[i]]
    tmp_inds[i] = tmp_inds[i][tmp_inds[i].rank(pct=True,axis=1,ascending=True) > 0.3]
    tmp_inds[i].fillna(0, inplace = True)
    tmp_rtn[i] = rtn[tmp_inds[i] != 0]
    tmp_rtn[i] = tmp_inds[i].apply(div, axis=1) * tmp_rtn[i]

MemoryError: Unable to allocate 120. MiB for an array with shape (4941, 3177) and data type int64

In [ ]:
del tmp_inds

In [ ]:
#weight 추출

#변수 설정
all_tmp_inds = [[] for _ in range(24)]
all_tmp_rtn = [[] for _ in range(24)]
all_tmp_signal = [[] for _ in range(24)]

#함수 설정
div = lambda x : x / x.sum()

#weight 추출
for i in notebook.tqdm(range(24)):
    all_tmp_inds[i] = re_mkt_cap[industry==inds[i]]
    all_tmp_inds[i].fillna(0, inplace = True)
    all_tmp_rtn[i] = rtn[all_tmp_inds[i] != 0]
    all_tmp_signal[i] = all_tmp_inds[i].apply(div, axis=1) * all_tmp_rtn[i]

산업군 24개가 있어
->> t-1 mkt_cap 상위 30% -> VW 투자 ( 24개 ) 
->> 3개씩 EW 투자한 포트폴리오 8개(Q1, Q2 ~ Q8)

In [ ]:
#산업별 투자수익률 넣을 빈 데이터 프레임 생성
temp = pd.DataFrame(index=re_mkt_cap.index, columns=inds)

for i in range(24):
    temp[inds[i]] = tmp_rtn[i].sum(axis=1)

In [ ]:
real_rtn = pd.DataFrame(index=re_mkt_cap.index, columns=inds)
for i in range(24):
    real_rtn[inds[i]] = all_tmp_rtn[i].sum(axis=1)
    
#real_signal = pd.DataFrame(index=re_mkt_cap.index, columns=inds)
#for i in range(24):
#    real_signal[inds[i]] = all_tmp_signal[i].sum(axis=1)

In [ ]:
del all_tmp_rtn
del all_tmp_inds
del all_tmp_signal

In [ ]:
signal = makemonth(temp)
r_rtn = makemonth(real_signal)

In [ ]:
#분위 추출
def makedecile(x):
    return pd.qcut(x,8,labels=False,duplicates='drop')
decile = signal.apply(makedecile, axis=1)

In [ ]:
shift_decile = decile.shift(1)

In [ ]:
#분위별 수익률 추출
rtn_decile = [[] for _ in range(24)]
for i in range(8):
    rtn_decile[i] = r_rtn[shift_decile == i].mean(axis=1)

In [ ]:
#분위별 수익률 넣을 데이터프레임 만들기
Quantile = ["Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8"]
rtn_signal = pd.DataFrame(index=signal.index, columns = Quantile)

In [ ]:
#분위별 수익률 데이터프레임에 넣기
for i in range(8):
    rtn_signal[Quantile[i]] = rtn_decile[i]

In [ ]:
anomaly_return = rtn_signal['Q8'] - rtn_signal['Q1']
pd.DataFrame(anomaly_return.values.tolist(), index=anomaly_return.index)
anomaly_return=anomaly_return.apply(pd.Series)
anomaly_return.columns = ['10-1 portfolio']

In [ ]:
((anomaly_return + 1).cumprod() - 1).plot(figsize=(16,10))

In [ ]:
((rtn_signal + 1).cumprod() - 1).plot(figsize=(16,10))